In [125]:
import os,re,numpy,sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


In [127]:
path=os.getcwd()
folder = next(os.walk(path))[1]
for sample_folder in folder:
    if sample_folder.startswith('ZC4'):
        #print(sample_folder)
        goal_cazy = path+'/'+sample_folder+'/transcriptoma_cazys.txt'
        if os.path.isfile(goal_cazy):
            cazy_nodes = pd.read_table(goal_cazy,names=['cazy','node'],sep='\t',)
            cazy_nodes['node'] = cazy_nodes['node'].replace('.*:','',regex=True)
            cazy_nodes['node'] = cazy_nodes['node'].replace(',.*','',regex=True)
            
            files = next(os.walk(path+'/'+sample_folder))[2]
            df_list=list()
            for file_cov in files:
                if file_cov.startswith('D') and file_cov.endswith("_cov.txt"):
                    #print(file_cov)
                    day = file_cov.replace('_cov.txt','')
                    #print(day)
                    transc_cov = pd.read_table(path+'/'+sample_folder+'/'+file_cov,sep='\t', )
                    transc_cov['#ID'] = transc_cov['#ID'].str.split(',').str[1]
                    transc_cov['Total_reads'] = transc_cov.Plus_reads + transc_cov.Minus_reads
                    transc_cov_cazy = transc_cov.loc[:,['#ID','Length','Total_reads','Covered_percent']]
                    #transc_cov_cazy['DAY'] = day
                    
                    #print(cazy_nodes.head())
                    #print(transc_cov_cazy.head())
                    
                    sum_read_size=0
                    for index,row in transc_cov_cazy.iterrows():
                        sum_read_size+=(row['Total_reads'])/(row['Length']*10**-3) #passo 1 e 2
                    tpm_array=[]

                    for index,row in transc_cov_cazy.iterrows():
                        tpm = (row['Total_reads']*10**6)/(sum_read_size*(row['Length']*10**-3))
                        tpm_array.append(tpm)
                        
                    transc_cov_cazy[day] = tpm_array
                    #print(transc_cov_cazy['TPM'].sum())
                    merged_df = pd.merge(cazy_nodes, transc_cov_cazy, left_on = 'node', right_on = '#ID',)
                    #merged_df = merged_df.drop(['#ID'], axis=1)
                    merged_df = merged_df.drop(['#ID','Length','Total_reads','Covered_percent'], axis=1)
                    #print(merged_df)
                    
                    df_list.append(merged_df)        
    df = df_list[0]
    for df_ in df_list[1:]:
        df = df.merge(df_, how = "outer", on=("node","cazy"))                
    df.to_csv(sample_folder+".csv", sep='\t')
#print(df_list)



In [124]:
df = df_list[0]
for df_ in df_list[1:]:
    df = df.merge(df_, how = "outer", on=("node","cazy"))
#df.fillna(0,inplace=True)
#df_long=df.melt(id_vars=['node','cazy'])
#df_long.head(10)

df
                    

,cazy,node,D1,D3,D7,D15,D30,D64,D78,D99
0,AA7,DIU68_04680,835.043919,0.000000,0.000000,174.038274,346.397638,0.000000,0.000000,0.000000
1,AA3,DIU68_05500,1510.255213,1012.348654,1124.452438,786.911338,626.492604,237.331989,228.744037,0.000000
2,AA7,DIU68_19300,0.000000,0.000000,148.534481,176.709915,0.000000,0.000000,256.835410,307.162037
3,AA6,DIU68_20345,0.000000,0.000000,201.582510,479.641197,477.327698,2169.892471,174.281171,3334.902112
4,AA3,DIU68_13070,0.000000,0.000000,67.596530,0.000000,160.061983,121.271436,58.441590,279.572632
5,AA3,DIU68_13790,523.882567,0.000000,45.888701,0.000000,0.000000,41.163272,0.000000,379.582354
6,AA7,DIU68_21215,824.361054,122.796141,144.417320,0.000000,683.932224,453.410367,561.861686,0.000000
7,GH5,DIU68_04700,0.000000,0.000000,90.550432,0.000000,857.658324,162.451843,0.000000,0.000000
8,GH5,DIU68_06450,0.000000,67.994557,39.983308,0.000000,94.676568,143.463965,103.704498,0.000000
9,GH5,DIU68_11550,0.000000,141.502187,83.208505,0.000000,0.000000,298.560143,791.330722,344.142233
